# SISTEMA DE RECOMENDACIÓN DE EJERCICIOS DE PROGRAMACIÓN

## 1. COMPRENCIÓN DEL PROBLEMA

Dentro de la Universidad Andrés Bello, los estudiantes de carreras de ingeniería deben cursar la asignatura de "Introducción a la Programación", ya que necesitan tener conocimientos básicos, medios o avanzados de programación, dependiendo de la carrera que estén cursando y en qué año van.

La problemática surge cuando ciertos estudiantes reprueban o sacan notas mínimas en las pruebas y trabajos de la asignatura. Siendo esta la primera asignatura de programación de ciertas carreras, puede marcar seriamente el desempeño del estudiante en el futuro, llegando a provocar que algunos tengan deficiencias en conocimientos esenciales para sus carreras.

Además, los estudiantes a menudo carecen de orientación personalizada para su aprendizaje, lo que puede resultar en un bajo rendimiento en los métodos de estudio, ocasionando un bajo conocimiento o baja nota en la asignatura.

Por tanto, este proyecto busca desarrollar e implementar un sistema de recomendación de ejercicios de programación para estudiantes universitarios de ingeniería de la Universidad Andrés Bello. Este sistema personalizará las sugerencias de ejercicios según el nivel de habilidad, preferencias individuales y objetivos de aprendizaje de cada estudiante.

## 2. COMPRENCIÓN DE LOS DATOS

#### CATALOGO

| Campo         | Descripción                                                                                                                                               | Tipo  |   Valores     | 
|---------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------|-------|---------------|
| hito          | Conjunto de ejercicios                                                                                                                                    | int   | [1,2,3,4]     |
| oid           | Identificador del ejercicio                                                                                                                               | str   | *             |
| nombre        | Nombre del ejercicio                                                                                                                                      | str   | *             |
| s1            | Nivel 4 en Algoritmos - Algoritmos Recursivos                                                                                                             | int   | binario [0,1] |
| s2            | Nivel 3 en Algoritmos - Algoritmos que combinan varias estructuras de datos                                                                               | int   | binario [0,1] |
| s3            | Nivel 2 en Algoritmos - Algoritmos que presentan información utilizando una estructura de datos                                                           | int   | binario [0,1] |
| s4            | Nivel 1 en Algoritmos - Algoritmos que no requieren una estructura de datos                                                                               | int   | binario [0,1] |
| k1            | Conocimientos en definir una clase con atributos y métodos.                                                                                               | int   | binario [0,1] |
| k2            | Conocimientos en agregar, quitar y buscar elementos dentro de una estructura de datos (cadenas o listas).                                                 | int   | binario [0,1] |
| k3            | Conocimientos en definir un bucle y determinar sus características en función del problema.                                                               | int   | binario [0,1] |
| k4            | Conocimientos en definir variables, usar operadores de comparación, construir expresiones condicionales, etc.                                             | int   | binario [0,1] |
| skill         | un escalar que indica la habilidad del estudiante                                                                                                         | int   | (0, 12)       |
| knowledge     | un escalar que indica el conocimiento del estudiante                                                                                                      | int   | (0, 12)       |
| complexity    | un escalar que indica la complejidad, número binario de 8 bits resultante de unir s1,s2,s3,s4,k1,k2,k3,k4                                                 | int   | binario [0,1] |
| item          | Código alfabético asociado a la complejidad, combina habilidad (4 bits en ASCII) con conocimiento (4 bits en ASCII): 0->a,1->b,2->c, etc.                 | str   | *             |


#### DATA_A

| Campo         | Descripción                                                                                                                              | Tipo  | Valores    | 
|---------------|------------------------------------------------------------------------------------------------------------------------------------------|-------|------------|
| rut           | Número de identificación ficticio del estudiante                                                                                         | int   | continuo   |
| usuario       | anonimo                                                                                                                                  | *     | *          |
| correo        | anonimo                                                                                                                                  | *     | *          |
| nombre        | anonimo                                                                                                                                  | *     | *          |
| nrc           | nrc del curso en donde el estuiante vio la asignatura                                                                                    | int   | continuo   |
| profesor      | anonimo                                                                                                                                  | *     | *          |
| i-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| f-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| t-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| r-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| p-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| s-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| l-(1, 6)      | anonimo                                                                                                                                  | *     | *          |
| op-(1, 6)     | anonimo                                                                                                                                  | *     | *          |
| sv-(1, 6)     | anonimo                                                                                                                                  | *     | *          |
| lt            | anonimo                                                                                                                                  | *     | *          |
| tt            | anonimo                                                                                                                                  | *     | *          |
| pt            | anonimo                                                                                                                                  | *     | *          |
| game_score    | anonimo                                                                                                                                  | *     | *          |
| score         | anonimo                                                                                                                                  | *     | *          |
| score_a       | anonimo                                                                                                                                  | *     | *          |
| score_p       | anonimo                                                                                                                                  | *     | *          |
| score_d       | anonimo                                                                                                                                  | *     | *          |
| score_s       | anonimo                                                                                                                                  | *     | *          |
| tema2         | anonimo                                                                                                                                  | *     | *          |
| tema3         | anonimo                                                                                                                                  | *     | *          |
| tema4         | anonimo                                                                                                                                  | *     | *          |
| tema5         | anonimo                                                                                                                                  | *     | *          |
| tema6         | anonimo                                                                                                                                  | *     | *          |
| tema7         | anonimo                                                                                                                                  | *     | *          |
| tema8         | anonimo                                                                                                                                  | *     | *          |
| tema9         | anonimo                                                                                                                                  | *     | *          |
| hito-(1, 4)   | anonimo                                                                                                                                  | *     | *          |
| seq-(1, 4)    | anonimo                                                                                                                                  | *     | *          |
| c-(1, 4)      | anonimo                                                                                                                                  | *     | *          |
| e-(1, 4)      | anonimo                                                                                                                                  | *     | *          |
| clase         | anonimo                                                                                                                                  | *     | *          |
| exitosos      | anonimo                                                                                                                                  | *     | *          |
| fallidos      | anonimo                                                                                                                                  | *     | *          |
| envios        | anonimo                                                                                                                                  | *     | *          |
| videos        | anonimo                                                                                                                                  | *     | *          |
| programa      | anonimo                                                                                                                                  | *     | *          |
| sol-(1, 4)    | anonimo                                                                                                                                  | *     | *          |
| tarea1        | anonimo                                                                                                                                  | *     | *          |
| tarea2        | anonimo                                                                                                                                  | *     | *          |
| controles     | anonimo                                                                                                                                  | *     | *          |
| np            | anonimo                                                                                                                                  | *     | *          |
| examen        | anonimo                                                                                                                                  | *     | *          |
| final         | anonimo                                                                                                                                  | *     | *          |
| estado        | anonimo                                                                                                                                  | *     | *          |

#### DATA_A_2021

| Campo             | Descripción                                                                                      | Tipo   | Valores       |
|-------------------|--------------------------------------------------------------------------------------------------|--------|---------------|
| hito-(1, 4)       | Son los puntajes obtenidos al realizar los ejercicios referentes de ese hito                     | int    | [1,2,3,4]     |
| seq-(1, 4)        | Secuencia de Complejidad de Ejercicios (alfabética) del Hito-n                                   | str    | *             |
| c-(1, 4)          | Complejidad Numérica Total del Hito-n                                                            | str    | *             |
| nrc               | NRC del curso                                                                                    | int    | continuos     |
| exitosos          | Número de envíos exitosos                                                                        | int    | *             |
| fallidos          | Número de envíos fallidos                                                                        | int    | *             |
| envidos           | Número total de envíos                                                                           | int    | *             |
| e-(0, 52)         | Ejercicio que realizó la persona dentro de la plataforma                                         | int    | binario [0,1] |
| programa          | ID de la carrera que está cursando el estudiante en la UNAB                                      | str    | *             |
| sol1              | Nota del estudiante al finalizar el semestre                                                     | float  | [1.0, 7.0]    |
| id                | ID del registro enviado                                                                          | int    | continuos     |

In [3]:
# MODULOS
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# RUTAS
DIR_DATA_ORIGINAL = './data/original'
DIR_DATA_LIMPIA = './data/cleaning'

# El catalogo se conforma por aquellos ejercicios de programacion que estaban disponibles en la pagina web.
df_catalogo = pd.read_csv(f"{DIR_DATA_ORIGINAL}/catalogo.csv", sep=";", encoding="latin1")
# La data_a son resgistros generales de los estudiantes durantes el semestre, es deicr, datos del curso en general.
df_data_a = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a.csv", sep=";", encoding="latin1")
# La data_a_2021 son resgitros globales de los estudiantes al tratar de realizar los ejercicios de la pagina web.
df_data_a_2021 = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a_2021.csv", sep=";", encoding="latin1")

# FUNCIONES
def show_dataframe_dimension(dataframe: pd.DataFrame) -> None:
    # Muestra las dimensiones del dataframe, tanto numero de registros guardados y sus atributos.
    print(f'[+] Número de Filas: {dataframe.shape[0]}')
    print(f'[+] Número de Columnas: {dataframe.shape[1]}')
    print(f'[*] Nombre de las columnas: {list(dataframe.columns)}')
    print()

def show_column_data_types(dataframe: pd.DataFrame)-> None:
    # Muestra los tipos de datos que se guardan en cada columnas de atributos del dataframe.
    print(f"[+] Columnas númericas: {dataframe.select_dtypes(include=['int', 'float']).columns}\n")
    print(f"[+] Columnas categoricas: {dataframe.select_dtypes(include=['object']).columns}\n")

def show_missing_data(dataframe: pd.DataFrame) -> None:
    # Muestra las cantidades de datos faltantes o vacios persisten dentro del dataframe. 
    data_faltante = dataframe.isna().sum().where(lambda x: x > 0).dropna().reset_index()
    data_faltante.columns = ['columna', 'datos_faltantes']
    data_faltante['porcentaje'] = (data_faltante['datos_faltantes'] / dataframe.shape[0]) * 100
    print(data_faltante.sort_values(by='porcentaje'))

def rename_dataframe_column(nombre_columna: str) -> str:
    # Renombra los atributos en minusculas y elimina los espacios de un dataframe
    return nombre_columna.replace(' ', '_').replace('-', '_').lower()

def show_correlation_columns(dataframe: pd.DataFrame, column_feature: str, top_n_columns: int = 20) -> None:
    # Muestra el porcentaje de correlacion de los atributos en base a la variable independiente, en este caso, column_feature.
    corr_matrix = dataframe.select_dtypes(include=['int', 'float']).corr()
    print(corr_matrix[column_feature].sort_values(ascending=False)[:top_n_columns])


In [ ]:
### DIMENSIONES DE LOS DATASETS

# CATALOGO
print(f'Dataframe: catalogo.csv')
show_dataframe_dimension(df_catalogo)

# DATA_A
print(f'Dataframe: data_a.csv')
show_dataframe_dimension(df_data_a)

# DATA_A_2021
print(f'Dataframe: data_a_2021.csv')
show_dataframe_dimension(df_data_a_2021)

In [ ]:
# VISUALIZACION: CATALOGO
df_catalogo.head()

In [ ]:
# VISUALIZACION: DATA_A
df_data_a.head()

In [ ]:
# VISUALIZACION: DATA_A_2021
df_data_a_2021.head()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: CATALOGO
df_catalogo.info()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: DATA_A
df_data_a.info()

In [ ]:
# INFORMACIÓN DE LAS VARIABLES DEL DATASET: DATA_A_2021
df_data_a_2021.info()

In [ ]:
# DATOS FALTANTES: CATALOGO
show_missing_data(df_catalogo)
# Observacion: El dataset de los ejercicios no contempla con datos vacios.Cuenta con 53 ejercicios.

In [ ]:
# DATOS FALTANTES: DATA_A
show_missing_data(df_data_a)
# Observacion: Para el dataset 'data_a.csv' tiene multiples columnas con datos faltantes, la mayoria tienen que ver con los ejercicios que realizaron los estudiantes. 
# Por tanto, se puede suponer que los estudiantes hacen mas ejercicios de los Hito 1 y 2, siendo que la mayoria de columnas del hito 3 y 4 tienen un 100% de datos faltantes. 

In [ ]:
# DATOS FALTANTES: DATA_A_2021
show_missing_data(df_data_a_2021)
# Observacion: El dataset 'data_a_2021.csv' tienen el mismo problema que el anterior, se nota que existen semejanzas de atributos con mayores porcentajes de datos faltantes. 
# Las columnas que se repiten como Seq1, Seq2, Seq3 y Seq4, representan la secuencia de ejercicios realizados en cada hito por los estudiantes, es decir, 
# Al tener Seq3 y Seq4 todas sus datos vacios en esas columnas, se puede afirmar que los estudiantes solo realizaron ejercicos del Hito 1, en su mayoria, y algunos del Hito 2.

In [ ]:
# DATOS ESTADISTICOS: CATALOGO
df_catalogo.describe().T

In [ ]:
# DATOS ESTADISTICOS: DATA_A
df_data_a.describe().T

In [ ]:
# DATOS ESTADISTICOS: DATA_A_2021
df_data_a_2021.describe().T

In [ ]:
# CORRELACIÓN DE VARIABLES: CATALOGO
show_correlation_columns(df_catalogo, 'complexity')
# Nota: 
# Teniendo en cuenta que, si se tuviera que relacionar la complejidad del ejercicio que el estudiante realiza, su puntaje deberia estar influenciado por esta correlacion...
# Teniendo en cuenta, que las variables 'Skill' y 'Knowledge' vienen de las variables one-hot-encoded, tienen mucha correlacion, pero la complejidad esta muy influenciada por la habilidad o 'Skill' tenga el ejercicio.

In [ ]:
# CORRELACIÓN DE VARIABLES: DATA_A
show_correlation_columns(df_data_a, 'final')
# Observacion:
# Las variables 'final' y 'np' estan estrechamente correlacionados, esto se puede deber ya que se representan la nota final y nota de presentacion del estudiante al final del semestre.
# Tambien se puede observar como las solemnes 4 y 3 estan igualmente correlacionadas con la nota final del estudiante. Sin embargo, esto es ilogico, ya que si los estudiantes quisieran 
# mejorar su rendimiento academico y mejorar sus notas para eximir la asignatura, deberian realizar ejercicios de los hitos 3 y 4. Asi mismo, reducirian las probabilidades de reprobar.

In [ ]:
# CORRELACIÓN DE VARIABLES: DATA_A_2021
show_correlation_columns(df_data_a_2021, 'sol1')
# Nota:
# Al igual que el dataset anterior, solo se muestra ejercicios y la variable a correlacionar, en este caso, seria 'sol1' que representa el puntaje de la solemne 1 que saco el estudiante. 
# En este dataset, solo muestra los ejercicios que mas fluyo en la obtencion del puntaje de la 'sol1'. 

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: CATALOGO  
df_catalogo.hist(figsize=(22, 20))
plt.show()

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: DATA_A
df_data_a.hist(figsize=(30,30))
plt.show()

In [ ]:
# DISTRIBUCIÓN DE LOS DATOS: DATA_A_2021
df_data_a_2021.hist(figsize=(30, 25))
plt.show()

In [ ]:
df_catalogo = df_catalogo[['nombre', 'hito', 'skill', 'knowledge', 'complexity']].sort_values(by=["hito", "complexity"], ascending=True)
df_catalogo.head(10)

In [25]:
df_data_a = df_data_a.sort_values(by='rut', ascending=True).reset_index(drop='index')
df_data_a = df_data_a[['rut','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'e1', 'e2', 'e3', 'e4', 'sol1', 'sol2', 'sol3', 'sol4','estado']]

df_data_a_2021 = df_data_a_2021.sort_values(by='id', ascending=True).drop(labels=['id'], axis=1)
df_data_a_2021['rut'] = list(range(df_data_a.shape[0], df_data_a.shape[0] + df_data_a_2021.shape[0]))
df_data_a_2021 = df_data_a_2021[['rut', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'p1', 'p2', 'p3', 'p4', 'sol1']]
df_data_a_2021.columns = ['rut', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'e1', 'e2', 'e3', 'e4', 'sol1']


df_data = pd.concat([df_data_a, df_data_a_2021], ignore_index=True)
df_data.columns = ['rut','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4', 'sol1', 'sol2', 'sol3', 'sol4','estado']

In [ ]:
df_data.head(10)

In [ ]:
show_missing_data(df_data)

In [27]:
# FUENTE: https://programacionacademica.unab.cl/

df_data['programa'] = df_data['programa'].replace({
    'UNAB12100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22115':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22510':'INGENIERIA INDUSTRIAL',
    'UNAB12510':'INGENIERIA INDUSTRIAL',
    'UNAB22505':'INGENIERIA INDUSTRIAL',
    'UNAB21503':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB11500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB21500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB12210':'INGENIERIA CIVIL INFORMATICA',
    'UNAB18000':'BACHILLERATO EN CIENCIAS',
    'UNAB16507':'ED. CONTINUA - ECONOMIA Y NEG',
    'UNAB35636':'MAG. EN INGENIERIA INDUSTRIAL',
    'UNAB10100':'PROG. DE INTERCAMBIO',
})

In [ ]:
df_data['programa'].value_counts()

### GRAFICOS: catalogo.csv

In [ ]:
# Crear la figura y los ejes
fig, axs = plt.subplots(2, 2, figsize=(12, 10))

# Ajustar la disposición de los subplots
fig.tight_layout(pad=5.0)

# Graficar el swarmplot
sns.stripplot(data=df_catalogo, x='knowledge', y='skill', hue='hito', palette="muted", dodge=True, ax=axs[0, 0])
axs[0, 0].set_title('Swarmplot: Knowledge vs Skill by Hito')

# Ocultar el cuarto subplot (axs[1, 1]) si no se usa
axs[0, 1].axis('off')

# Graficar el histplot para 'skill'
sns.histplot(data=df_catalogo, x='skill', kde=True, ax=axs[1, 0])
axs[1, 0].set_title('Histogram of Skill')

# Graficar el histplot para 'knowledge'
sns.histplot(data=df_catalogo, x='knowledge', kde=True, ax=axs[1, 1])
axs[1, 1].set_title('Histogram of Knowledge')

# Mostrar la figura
plt.show()

### GRAFICOS: df_data

In [ ]:
# Crear histogramas de todas las columnas
axes = df_data.hist(figsize=(30, 25))
# Mostrar la figura
plt.show()

In [ ]:
carreras = df_data['programa'].value_counts()
fig, ax = plt.subplots(figsize=(12, 5))  
sns.barplot(x=carreras.values, y=carreras.index, orient='h', hue=carreras.index, palette='Set2', dodge=False, ax=ax)

for container in ax.containers:
    ax.bar_label(container, fontsize=10)

plt.title('Total de estudiantes registrados por carrera', fontsize=16)
plt.xlabel('Carreras')
plt.ylabel('Número total de estudiantes participantes')
plt.show()

In [ ]:
rangos = [(1.0, 2.0), (2.0, 3.0), (3.0, 4.0), (4.0, 5.0), (5.0, 6.0), (6.0, 7.0), (7.0, 7.1)]

def contar_personas_en_rango(df, rango):
    rango_inferior, rango_superior = rango
    return df[(df['sol1'] >= rango_inferior) & (df['sol1'] < rango_superior)].shape[0]

conteo_por_rango = []
for rango in rangos:
    conteo = contar_personas_en_rango(df_data, rango)
    conteo_por_rango.append((rango, conteo))

# Preparar los datos para el gráfico
rangos_str = [f"{rango[0]:.1f}-{rango[1]:.1f}" for rango in rangos]
conteos = [conteo for rango, conteo in conteo_por_rango]

# Crear el gráfico de barras
plt.figure(figsize=(10, 6))
bars = plt.bar(rangos_str, conteos, color='skyblue')

# Agregar etiquetas sobre cada barra
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2, yval, int(yval), ha='center', va='bottom')

plt.xlabel('Rangos de sol1')
plt.ylabel('Número de personas')
plt.title('Número de personas en cada rango de sol1')
plt.xticks(rotation=45)
plt.show()

In [ ]:
programa_df = pd.DataFrame({'Aprobados': df_data[df_data['sol1'] >= 4.0].groupby('programa').size(), 'Reprobados': df_data[df_data['sol1'] < 4.0].groupby('programa').size()})
programa_df.fillna(0.0, inplace=True)
programa_df = programa_df.astype(int)   # Aseguramos que todos los valores sean enteros
y = np.arange(len(programa_df.index))   # las ubicaciones de las etiquetas
height = 0.25                           # la altura de las barras
multiplier = 0

fig, ax = plt.subplots(figsize=(14, 6))

for attribute, measurement in programa_df.to_dict().items():
    offset = height * multiplier
    rects = ax.barh(y + offset, measurement.values(), height, label=attribute)
    ax.bar_label(rects, padding=3)
    multiplier += 1

# Añadir algunos textos para etiquetas, título y etiquetas personalizadas del eje x, etc.
ax.set_xlabel('Cantidad')
ax.set_title('Estudiantes aprobados y reprobados por programa (Solemne 1)')
ax.set_yticks(y + height / 2)
ax.set_yticklabels(programa_df.index)
ax.legend(loc='lower right')
ax.set_xlim(0, programa_df.max().max() + 10)  # Ajuste de límites del eje x

plt.show()

## 3. PREPARACIÓN DE LOS DATOS

In [46]:
# MODULOS
import warnings
import numpy as np 
import pandas as pd
warnings.filterwarnings('ignore')

# RUTAS
DIR_DATA_ORIGINAL = './data/original'
DIR_DATA_LIMPIA = './data/cleaning'

#### EJERCICIOS

In [47]:
# CARGAR DATASETS DE CATALOGO DE EJERCICIOS
catalogo = pd.read_csv(f"{DIR_DATA_ORIGINAL}/catalogo.csv", sep=";", encoding="latin1")

hitos = catalogo['hito'].apply(lambda x: format(x, '04b') if 1 <= x <= 4 else None)
# Crear columnas h1, h2, h3, h4 a partir de 'aux'
catalogo['h1'] = hitos.apply(lambda x: x[0] if x else None)  # Primer bit (más significativo)
catalogo['h2'] = hitos.apply(lambda x: x[1] if x else None)  # Segundo bit
catalogo['h3'] = hitos.apply(lambda x: x[2] if x else None)  # Tercer bit
catalogo['h4'] = hitos.apply(lambda x: x[3] if x else None)  # Cuarto bit (menos significativo)

catalogo = catalogo[['oid', 'nombre', 'h1', 'h2', 'h3', 'h4','s1','s2','s3', 's4', 'k1', 'k2', 'k3', 'k4', 'hito', 'skill', 'knowledge', 'complexity']] 

# catalogo['complexity'] = (
#     catalogo[['s1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4']]
#     .astype(str)
#     .agg(''.join, axis=1)
#     .apply(lambda x: int(x.zfill(8), 2))
# )

catalogo['complexity12'] = (
    catalogo[['h1', 'h2', 'h3', 'h4', 's1', 's2', 's3', 's4', 'k1', 'k2', 'k3', 'k4']]
    .astype(str)
    .agg(''.join, axis=1)
    .apply(lambda x: int(x.zfill(12), 2))
)

# LA VARAIBLE IDX CORRESPONDE AL ORDEN ANTES DEL ORDENAMIENTO, ESTO ES PARA LA REFERENCIA DEL DATAFRAME DE REGISTROS DE LOS ALUMNOS (DATA_A_2021.CSV)
catalogo.to_csv(f'{DIR_DATA_LIMPIA}/catalogo.csv', index=False)

In [37]:
catalogo.sort_values(by='complexity', ascending=True).head(10)

,oid,nombre,h1,h2,h3,h4,s1,s2,s3,s4,k1,k2,k3,k4,hito,skill,knowledge,complexity,complexity12
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257
25,89f44e7f5842479fb283e43c52ce067b,Ordenar tres números,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257
23,80c61dae74fa4915bf272ab17dfa62ff,Conversor de Decimal a Binario,0,0,0,1,0,0,0,0,0,0,1,0,1,0,2,2,258
17,718578451f3f4eca87437cadfe98d688,Suma de los N primeros números naturales,0,0,0,1,0,0,0,0,0,0,1,0,1,0,2,2,258
26,8f24397e36034cccb71e9d578975c33d,Contestador Automático,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259
42,d8395f43e4a1454d90346ac5a1ba561a,Números Amigos,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259
32,ac7382763e484d37908da54c076f7577,Cajero Automático Nivel 1,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259
3,171b5e86d4fb47268f2692587fbec073,Números Primos,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259
29,a3963220090f4e50a266ce53d33b9841,Aprobación de Créditos,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259


#### DATA_A & DATA_A_2021

In [38]:
# DATA_A
df_data_a = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a.csv", sep=";", encoding="latin1")

# DATA_A_2021
df_data_a_2021 = pd.read_csv(f"{DIR_DATA_ORIGINAL}/dataset_a_2021.csv", sep=";", encoding="latin1")

# ORDENAR SEGUN EL IDENTIFICADOR ANONIMO DEL ALUMNO (rut)
df_data_a = df_data_a.sort_values(by='rut', ascending=True)

# ORDENAR SEGUN EL IDENTIFICADOR ANONIMO DEL ALUMNO (id)
df_data_a_2021 = df_data_a_2021.sort_values(by='id', ascending=True).drop(labels=['id'], axis=1)

# EN ESTE CASO, SE JUNTARAN LOS DOS DATASETS EN UNO PERO PARA ELLO HAY QUE CORREGIR LOS IDENTIFICADORES DE LOS ALUMNOS, YA QUE SON IMAGINARIOS SE PUEDE CORRER EL RANGO DE RUTS
df_data_a_2021['rut'] = list(range(df_data_a.shape[0], df_data_a.shape[0] + df_data_a_2021.shape[0])) # 467 hasta (467+839)-1 

# DATA_A
df_data1 = df_data_a[['rut','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'e1', 'e2', 'e3', 'e4', 'sol1', 'sol2', 'sol3', 'sol4']]
df_data1.columns = ['rut','programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4', 'sol1', 'sol2', 'sol3', 'sol4']

# DATA_A_2021
df_data2 = df_data_a_2021[['rut', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'p1', 'p2', 'p3', 'p4', 'sol1']]
df_data2.columns = ['rut', 'programa', 'hito1', 'hito2', 'hito3', 'hito4', 'exitosos', 'fallidos', 'ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4', 'sol1']

# PARA EL DATASET DATA_A_2021, SE TIENEN QUE HACER UNAS MODIFICACIONES EN LOS DATOS CONVIRTIENDO AQUELLOS CODIGOS DE LOS PROGRAMAS EN NOMBRES, PARA TENER UNA SEMEJANZA EN EL DATASET COMBINADO. 
df_data2['programa'] = df_data2['programa'].replace({
    'UNAB12100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22115':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22100':'INGENIERIA CIVIL INDUSTRIAL',
    'UNAB22510':'INGENIERIA INDUSTRIAL',
    'UNAB12510':'INGENIERIA INDUSTRIAL',
    'UNAB22505':'INGENIERIA INDUSTRIAL',
    'UNAB21503':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB11500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB21500':'INGENIERIA EN COMPUTACION E INFORMATICA',
    'UNAB12210':'INGENIERIA CIVIL INFORMATICA',
    'UNAB18000':'BACHILLERATO EN CIENCIAS',
})

# ADEMAS, SE TIENE SE ELIMINARAN ALGUNOS OUTLAYERS DE ESTUDIANTES DE CARRERAS NO RELACIONADAS DIRECTAMENTE CON LA ASIGNATURA U OTROS QUE CURSAN ESTUDIOS SUPERIORES AL PREGRADO.
# 'UNAB16507':'ED. CONTINUA - ECONOMIA Y NEG', # 'UNAB10100':'PROG. DE INTERCAMBIO', # 'UNAB35636':'MAG. EN INGENIERIA INDUSTRIAL',
df_data2.drop(df_data2.query(" `programa`=='UNAB16507' or `programa`=='UNAB35636' or `programa`=='UNAB10100' ").index, inplace=True)

# RECHAZADAS
df1 = df_data1.copy()
df1.drop(df1.query("`hito1` < 1 and `hito2` < 1 and `hito3` < 1 and `hito4` < 1").index, inplace=True)
df1.drop(df1.query("`sol1` < 4.0 and `sol2` < 4.0 and `sol3` < 4.0 and `sol4` < 4.0").index, inplace=True)

df2 = df_data2.copy()
df2.drop(df2.query("`hito1` < 1 and `hito2` < 1 and `hito3` < 1 and `hito4` < 1").index, inplace=True)
df2.drop(df2.query("`sol1` < 4.0").index, inplace=True)

dataset_rechazadas = pd.concat([df_data1.drop(df1.index), df_data2.drop(df2.index)])
dataset_rechazadas['hito1'] = dataset_rechazadas['hito1'].astype(int)
dataset_rechazadas['hito2'] = dataset_rechazadas['hito2'].astype(int)
dataset_rechazadas.to_csv(f'{DIR_DATA_LIMPIA}/dataset_rechazadas.csv', index=False)


# FINALMENTE, TODOS LOS DATOS DE LOS REGISTROS DE LOS ESTUDIANTES SE REUNIRAN DETRO DE 'DATASET'
dataset = pd.concat([df1, df2])
dataset['hito1'] = dataset['hito1'].astype(int)
dataset['hito2'] = dataset['hito2'].astype(int)
dataset.to_csv(f'{DIR_DATA_LIMPIA}/dataset.csv', index=False)
dataset.head()

,rut,programa,hito1,hito2,hito3,hito4,exitosos,fallidos,ejercicios_hito1,ejercicios_hito2,ejercicios_hito3,ejercicios_hito4,sol1,sol2,sol3,sol4
283,0,INGENIERIA INDUSTRIAL,18,3,0,0,15,24,:25:17:4:26:29:23:0:22:10:3:42:18,:6:1:44,NaN,NaN,6.2,6.8,5.1,6.0
393,1,INGENIERIA CIVIL INDUSTRIAL,14,0,0,0,10,6,:25:17:4:26:29:23:0:22:10:3,NaN,NaN,NaN,7.0,6.9,7.0,7.0
99,2,INGENIERIA CIVIL INDUSTRIAL,17,0,0,0,11,20,:25:17:4:26:29:23:3:42:32:18:35,NaN,NaN,NaN,5.6,6.0,4.5,5.4
59,3,INGENIERIA CIVIL INDUSTRIAL,1,0,0,0,7,17,:25,NaN,NaN,NaN,3.9,4.7,3.7,4.1
284,4,INGENIERIA EN COMPUTACION E INFORMATICA,19,2,0,0,13,18,:25:17:4:29:23:0:22:3:42:32:13,:1:44,NaN,NaN,4.6,6.6,4.7,5.3


#### PRUEBA DIAGNOSTICO (DATA_A)

In [39]:
df_prueba_diagnostico = df_data_a[[
    'rut',
    't1', 'r1', 'p1',
    't2', 'r2', 'p2',
    't3', 'r3', 'p3',
    't4', 'r4', 'p4',
    't5', 'r5', 'p5',
    't6', 'r6', 'p6',
    'op1', 'op2', 'op3', 'op4', 'op5', 'op6',
    'pt', 'game_score', 'score', 'score_a', 'score_p', 'score_d', 'score_s'
]]
df_prueba_diagnostico.to_csv(f'{DIR_DATA_LIMPIA}/prueba_diagnostico.csv', index=False)
df_prueba_diagnostico.head()

,rut,t1,r1,p1,t2,r2,p2,t3,r3,p3,...,op4,op5,op6,pt,game_score,score,score_a,score_p,score_d,score_s
283,0,64.301310,1,4,251.056409,3,10,224.745993,1,19,...,1,1,0,56,20.192022,8,5,4,2,3
393,1,97.170900,2,5,93.273640,2,9,108.991540,1,18,...,1,0,1,46,17.102511,8,4,4,2,3
99,2,85.596734,1,8,217.071180,7,16,150.406520,1,11,...,0,0,1,49,17.445343,8,5,4,3,2
59,3,57.776163,1,2,97.665937,1,4,187.063634,2,11,...,0,1,0,35,19.181718,6,3,3,1,2
284,4,32.519816,1,3,57.514397,2,5,75.568439,1,8,...,0,1,1,45,13.494646,10,6,4,3,3


In [40]:
# DATASET RECHAZADA
dataset_rechazadas.head()

,rut,programa,hito1,hito2,hito3,hito4,exitosos,fallidos,ejercicios_hito1,ejercicios_hito2,ejercicios_hito3,ejercicios_hito4,sol1,sol2,sol3,sol4
0,5,INGENIERIA EN COMPUTACION E INFORMATICA,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
1,7,INGENIERIA CIVIL INDUSTRIAL,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
2,16,INGENIERIA INDUSTRIAL,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0
109,34,INGENIERIA CIVIL INFORMATICA,0,0,0,0,0,31,NaN,NaN,NaN,NaN,6.2,5.5,5.6,5.8
3,44,INGENIERIA CIVIL INDUSTRIAL,0,0,0,0,0,0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0


#### DATASET

In [41]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 766 entries, 283 to 838
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rut               766 non-null    int64  
 1   programa          766 non-null    object 
 2   hito1             766 non-null    int64  
 3   hito2             766 non-null    int64  
 4   hito3             766 non-null    int64  
 5   hito4             766 non-null    int64  
 6   exitosos          766 non-null    int64  
 7   fallidos          766 non-null    int64  
 8   ejercicios_hito1  763 non-null    object 
 9   ejercicios_hito2  359 non-null    object 
 10  ejercicios_hito3  0 non-null      float64
 11  ejercicios_hito4  0 non-null      float64
 12  sol1              766 non-null    float64
 13  sol2              424 non-null    float64
 14  sol3              424 non-null    float64
 15  sol4              424 non-null    float64
dtypes: float64(6), int64(7), object(3)
memory usage

> NOTA:

> Los datos que se utlizaran para los experimentos y creacion del sistema de recomendacion seran los mostrados en el 'dataset'. Sin embargo, no todos los datos seran utiles para entrenar un recomendador de ejecicios, ya que existen algunas anomalias en los registros de los estudiantes que puede que afecte a las recomendaciones del modelo. Por ejemplo: estudiantes que no realizaron nada durante la asignatura y reprobaron; estudiantes que no realizaron nada durante la asignatura y aprobaron; estudiantes que hicieron pocos o varios ejercicios y reprobaron la asignatura. 

> Por tanto, se agrupara los registros de estudiantes con un minimo de exito para usarlo y medir como reacciona con aquellos que tuvieron poco exito durante la asignatura. 

In [42]:
def create_exercise_list(n_items: int, exercises_performed: list) -> list:
    # Retorna una lista binaria de n-elementos que representen los ejercicios que han realizado los estudiantes. Siendo 1 cuando lo a realizado, sino 0. 
    exercises_list = [0] * n_items
    if not exercises_performed:
        return exercises_list
    for exercise in exercises_performed:
        if exercises_list[exercise] < 1:
            exercises_list[exercise] += 1
    return exercises_list


def create_factorization_matrix(df_users: pd.DataFrame, df_items: pd.DataFrame) -> pd.DataFrame:
    # Retorna una matriz factorizacion de usuarios x ejercicios, toda esta binaria. 
    n_items = len(df_items)
    matrix_factorizacion = pd.DataFrame(columns=['e' + str(i) for i in range(n_items)])
    sequence_exercises = df_users[['ejercicios_hito1', 'ejercicios_hito2', 'ejercicios_hito3', 'ejercicios_hito4']]

    for index in range(len(sequence_exercises)):
        exercises = sequence_exercises.iloc[index]

        exercises1 = [int(element) for element in str(exercises.ejercicios_hito1).split(':')[1:] if element.isdigit()] # Hito 1 
        exercises2 = [int(element) for element in str(exercises.ejercicios_hito2).split(':')[1:] if element.isdigit()] # Hito 2 
        exercises3 = [int(element) for element in str(exercises.ejercicios_hito3).split(':')[1:] if element.isdigit()] # Hito 3 
        exercises4 = [int(element) for element in str(exercises.ejercicios_hito4).split(':')[1:] if element.isdigit()] # Hito 4 

        exercises_performed = create_exercise_list(n_items, exercises1+exercises2+exercises3+exercises4)
        matrix_factorizacion.loc[len(matrix_factorizacion)] = exercises_performed

    matrix_factorizacion['rut'] = df_users['rut'].values
    matrix_factorizacion = matrix_factorizacion[ ['rut'] + ['e' + str(i) for i in range(n_items)] ]
    return matrix_factorizacion

#### MATRICES DE FACTORIZACION

In [43]:
mf_dataset = create_factorization_matrix(dataset, catalogo)
mf_dataset.to_csv(f'{DIR_DATA_LIMPIA}/mf_dataset.csv', index=False)

mf_dataset_rechazadas = create_factorization_matrix(dataset_rechazadas, catalogo)
mf_dataset_rechazadas.to_csv(f'{DIR_DATA_LIMPIA}/mf_rechazadas.csv', index=False)

In [44]:
mf_dataset.head()

,rut,e0,e1,e2,e3,e4,e5,e6,e7,e8,...,e43,e44,e45,e46,e47,e48,e49,e50,e51,e52
0,0,1,1,0,1,1,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
1,1,1,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,1,0,1,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


#### RATIO DE EJERCICIOS

In [45]:
ratio_de_ejercicios = mf_dataset.sum().values.tolist()[1:]
resultados = [ n / len(mf_dataset) for n in ratio_de_ejercicios]
max_valor = max(resultados)
indice_valor = resultados.index(max_valor)

print(f'Cantidad de veces hechos (Ejercicios): {ratio_de_ejercicios}')
print(f'Cantidad de registros de alumnos: {len(mf_dataset)}')
print(f'Porcentaje de realizacion del ejercicio: {resultados}')
print(f'Ejercicio con mayor participacion: {max_valor} - {indice_valor} - {ratio_de_ejercicios[indice_valor]}')

catalogo['ratio'] = resultados
catalogo.head(10)

Cantidad de veces hechos (Ejercicios): [603, 275, 17, 542, 733, 0, 277, 195, 21, 42, 599, 38, 69, 233, 0, 0, 0, 737, 389, 0, 0, 37, 650, 590, 0, 730, 678, 0, 168, 653, 0, 4, 395, 11, 0, 347, 0, 184, 87, 0, 0, 0, 472, 0, 314, 0, 0, 0, 4, 0, 0, 0, 0]
Cantidad de registros de alumnos: 766
Porcentaje de realizacion del ejercicio: [0.7872062663185379, 0.35900783289817234, 0.022193211488250653, 0.7075718015665796, 0.9569190600522193, 0.0, 0.36161879895561355, 0.2545691906005222, 0.02741514360313316, 0.05483028720626632, 0.7819843342036553, 0.04960835509138381, 0.09007832898172324, 0.304177545691906, 0.0, 0.0, 0.0, 0.9621409921671018, 0.5078328981723238, 0.0, 0.0, 0.048302872062663184, 0.8485639686684073, 0.7702349869451697, 0.0, 0.9530026109660574, 0.8851174934725848, 0.0, 0.2193211488250653, 0.8524804177545692, 0.0, 0.005221932114882507, 0.5156657963446475, 0.014360313315926894, 0.0, 0.45300261096605743, 0.0, 0.2402088772845953, 0.11357702349869452, 0.0, 0.0, 0.0, 0.6161879895561357, 0.0, 0

,oid,nombre,h1,h2,h3,h4,s1,s2,s3,s4,k1,k2,k3,k4,hito,skill,knowledge,complexity,complexity12,ratio
0,0973dae0e1b74ab8baa8d94339ee3ae6,Cálculo del dígito verificador del rut,0,0,0,1,0,0,0,1,0,1,1,1,1,1,7,23,279,0.787206
1,16f619db31204ded9418136c4587ddd8,Calculadora Geométrica,0,0,1,0,0,0,0,1,0,0,0,1,2,1,1,17,529,0.359008
2,17022c9ceac94ec5b2e7bc934c7b2d6f,Subsecuencias de ADN,0,0,1,0,0,0,1,1,0,1,1,1,2,3,7,55,567,0.022193
3,171b5e86d4fb47268f2692587fbec073,Números Primos,0,0,0,1,0,0,0,0,0,0,1,1,1,0,3,3,259,0.707572
4,2437df93d3f44a87b00834072aeb1ab0,Nota Final,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257,0.956919
5,24b87304a1eb4e95856da7f574a66fe8,Validador de Expresiones Matemáticas,0,1,0,0,1,0,0,0,0,1,1,1,4,8,7,135,1159,0.000000
6,29f15ef8dc32426f945f64e28c910a57,Suma de los divisores de un número,0,0,1,0,0,0,0,1,0,0,1,1,2,1,3,19,531,0.361619
7,31ea1c1b12174428b5a67a6576627de9,Resolver un sistema de ecuaciones,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,257,0.254569
8,37e4f4a1e8174e9496d21b00d67fc8f1,Distancia Levenshtein,0,0,1,0,0,0,1,1,0,1,1,1,2,3,7,55,567,0.027415
9,3d7d9e64ac0846caadc08dccf2538f55,Validar Secuencias de ADN,0,0,1,0,0,0,1,0,0,1,1,1,2,2,7,39,551,0.054830


## 4. MODELAMIENTO

## 5. EVALUCIÓN

## 6. IMPLEMENTACIÓN